In [15]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import numpy as np


import plotly.graph_objs as go
import chart_studio.plotly as py
import cufflinks
from plotly.offline impordt iplot
cufflinks.go_offline()
# Set global theme
cufflinks.set_config_file(world_readable=True, theme='white')

In [28]:
# # Barrios del Leo
# url = "https://www.portalinmobiliario.com/arriendo/departamento/rm-metropolitana/las-condes/barrio-el-golf-o-centro-financiero-o-metro-escuela-militar-o-metro-manquehue---apumanque-o-sebastian-elcano-o-vaticano"

# # Ubele: las condes
# url = "https://www.portalinmobiliario.com/arriendo/departamento/rm-metropolitana/las-condes/barrio-el-golf-o-centro-financiero-o-metro-escuela-militar-o-metro-manquehue---apumanque-o-parque-arauco"

# Ubele: Vitacura
url = "https://www.portalinmobiliario.com/arriendo/departamento/rm-metropolitana/vitacura/borde-rio---casa-piedra-o-parque-bicentenario-o-jardin-del-este-o-pio-xi-o-estadio-manquehue"

In [29]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
req = requests.get(url=url, headers=headers)

soup = BeautifulSoup(req.content, "html.parser")

In [30]:
# all houses in page
dataset = soup.find_all("div", {"class": "rowItem item highlighted item--stack new"}) 
print("Viviendas en página: {}".format(len(dataset)) )

Viviendas en página: 49


In [31]:
# helper function
def find(string, character):
    """Return all required characters' positions"""
    return [i for i, letter in enumerate(string) if letter == character]

In [32]:
%%time

infoset = []
_continue = True
page = 1



while _continue == True:
    print("Página: {}, Viviendas en página: {}".format(page,len(dataset)) )
    for apartment in dataset:
        d = {}
        try:
            price = apartment.find("span", {"class": "price__fraction"}).text.strip()
        except:
            price = None

        if price != None:
            price = price.replace('.','')
            d['price'] = int(price)
            
            try:
                d['currency'] = apartment.find("span", {"class": "price__symbol"}).text.strip()
            except:
                d['currency'] = None
            
            try:
                atributes = apartment.find("div", {"class": "item__attrs"}).text.strip()
                indexes = find(atributes,'|')

                # Area
                m2 = atributes[0:indexes[0]].strip()
                d['m2'] = float(m2[:-10])

                # Rooms
                dorms = atributes[indexes[0]+1:indexes[1]].strip()
                if 'dormitorios' in dorms:
                    d['dorms'] = int(dorms[0:-12])
                else:
                    d['dorms'] = int(dorms[0:-11])

                # Bathrooms
                baths = atributes[indexes[1]+1:].strip()
                if 'baños' in baths:
                    d['baths'] = int(baths[0:-6])
                else:
                    d['baths'] = int(baths[0:-5])

            except:
                d['atributes'] = None
                d['m2'] = None
                d['dorms'] = None
                d['baths'] = None

            try:
                link_html = apartment.find("a", {"class": "item__info-link"})
                d['link'] = link_html.get('href')
            except:
                d['link'] = None

            infoset.append(d)

    try:
        next_nav = soup.find("li", {"class": "andes-pagination__button andes-pagination__button--next"})
        next_nav = next_nav.find("a", {"class": "andes-pagination__link prefetch"}) # one more level deep
        next_url = next_nav.get('href')
        req = requests.get( url=next_url, headers=headers )
        time.sleep(1)
        soup = BeautifulSoup(req.content, "html.parser")
        dataset = soup.find_all("div", {"class": "rowItem item highlighted item--stack new"})
        page += 1
    except:
        _continue = False
        print("Terminado")

    

Página: 1, Viviendas en página: 49
Página: 2, Viviendas en página: 49
Página: 3, Viviendas en página: 47
Página: 4, Viviendas en página: 50
Página: 5, Viviendas en página: 50
Página: 6, Viviendas en página: 50
Página: 7, Viviendas en página: 47
Página: 8, Viviendas en página: 50
Página: 9, Viviendas en página: 50
Página: 10, Viviendas en página: 50
Página: 11, Viviendas en página: 49
Página: 12, Viviendas en página: 50
Página: 13, Viviendas en página: 50
Página: 14, Viviendas en página: 50
Página: 15, Viviendas en página: 50
Página: 16, Viviendas en página: 50
Página: 17, Viviendas en página: 50
Página: 18, Viviendas en página: 50
Página: 19, Viviendas en página: 20
Terminado


In [33]:
# transformar la lista a dataFrame
houses_df = pd.DataFrame(infoset)

# Borrar columna atributes
houses_df.drop(columns =['atributes'], inplace=True, errors='ignore')

In [34]:
houses_df

,price,currency,m2,dorms,baths,link
0,60,UF,130.0,2.0,3.0,https://www.portalinmobiliario.com/arriendo/de...
1,1000000,$,120.0,3.0,2.0,https://www.portalinmobiliario.com/arriendo/de...
2,800000,$,110.0,3.0,2.0,https://www.portalinmobiliario.com/arriendo/de...
3,620000,$,80.0,3.0,2.0,https://www.portalinmobiliario.com/arriendo/de...
4,1000000,$,131.0,3.0,3.0,https://www.portalinmobiliario.com/arriendo/de...
...,...,...,...,...,...,...
860,650000,$,45.0,1.0,1.0,https://www.portalinmobiliario.com/MLC-5366977...
861,27,UF,78.0,2.0,2.0,https://www.portalinmobiliario.com/MLC-5341484...
862,830000,$,135.0,3.0,3.0,https://www.portalinmobiliario.com/MLC-5336714...
863,370000,$,45.0,1.0,1.0,https://www.portalinmobiliario.com/MLC-5307001...


In [35]:
houses_df.loc[houses_df['currency'] == '$'].sort_values(by='price')

,price,currency,m2,dorms,baths,link
813,290000,$,28.0,1.0,1.0,https://www.portalinmobiliario.com/arriendo/de...
92,340000,$,45.0,1.0,1.0,https://www.portalinmobiliario.com/MLC-5383338...
108,350000,$,30.0,2.0,1.0,https://www.portalinmobiliario.com/arriendo/de...
863,370000,$,45.0,1.0,1.0,https://www.portalinmobiliario.com/MLC-5307001...
852,380000,$,40.0,1.0,1.0,https://www.portalinmobiliario.com/arriendo/de...
...,...,...,...,...,...,...
686,2600000,$,200.0,4.0,3.0,https://www.portalinmobiliario.com/arriendo/de...
509,3000000,$,257.0,4.0,5.0,https://www.portalinmobiliario.com/MLC-5268753...
216,3200000,$,200.0,3.0,3.0,https://www.portalinmobiliario.com/arriendo/de...
439,3500000,$,260.0,3.0,4.0,https://www.portalinmobiliario.com/arriendo/de...


In [36]:
houses_df.to_excel("ubele_vitacura.xlsx")